In [2]:
import pandas as pd
import numpy as np

# group by


In [4]:
df = pd.read_csv('titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.shape

(891, 12)

In [17]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
df_groupby_sex = df.groupby('Sex')
df_groupby_sex

In [7]:
type(df_groupby_sex)

pandas.core.groupby.groupby.DataFrameGroupBy

In [9]:
df_groupby_sex.ngroups  #use ngroups attribute to get the number of groups

2

In [11]:
df_groupby_sex.groups #Use groups attribute to get groups object. Those integer numbers in the list are the row number.

{'female': Int64Index([  1,   2,   3,   8,   9,  10,  11,  14,  15,  18,
             ...
             866, 871, 874, 875, 879, 880, 882, 885, 887, 888],
            dtype='int64', length=314),
 'male': Int64Index([  0,   4,   5,   6,   7,  12,  13,  16,  17,  20,
             ...
             873, 876, 877, 878, 881, 883, 884, 886, 889, 890],
            dtype='int64', length=577)}

In [12]:
df_groupby_sex.size() #use size() method to compute and display group sizes

Sex
female    314
male      577
dtype: int64

In [14]:
#To preview groups, we can call first() or last() to preview the result with the first or last entry.
df_groupby_sex.first()


,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Sex,,,,,,,,,,,
female,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C
male,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,E46,S


In [15]:
df_groupby_sex.last()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Sex,,,,,,,,,,,
female,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",19.0,1,2,W./C. 6607,23.45,B42,S
male,891,0,3,"Dooley, Mr. Patrick",32.0,0,0,370376,7.75,C148,Q


In [16]:
#We can use get_group() method to retrieve one of the created groups
df_female = df_groupby_sex.get_group('female')
df_female.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,SibSp,Survived,Ticket
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1,1,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,0,1,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1,1,113803
8,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,0,1,347742
9,14.0,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,1,1,237736


# The “group by” process: split-apply-combine

- Aggregation: compute a summary statistic for each group. for example, sum, mean, or count.
- Transformation: perform some group-specific computations and return a like-indexed object. For example, standardize data within a group or replacing missing values within groups.
- Filtration: discard some groups, according to a group-wise computation that evaluates True or False. For example, discard data that belongs to groups with only a few members or filter out data based on the group sum or mean.

## Aggregation

In [18]:
df.groupby('Sex')['Age'].sum()

Sex
female     7286.00
male      13919.17
Name: Age, dtype: float64

In [19]:
df.groupby('Sex')['Age'].count()

Sex
female    261
male      453
Name: Age, dtype: int64

In [20]:
df.groupby('Sex')['Age'].agg(['max', 'min', 'count', 'median', 'mean','sum'])

,max,min,count,median,mean,sum
Sex,,,,,,
female,63.0,0.75,261,27.0,27.915709,7286.00
male,80.0,0.42,453,29.0,30.726645,13919.17


In [27]:
# custom aggregation function:
def categorize(x):
    m = x.mean()
    return True if m > 29 else False
df.groupby('Sex')['Age'].agg(['max', 'mean', categorize])

,max,mean,categorize
Sex,,,
female,63.0,27.915709,False
male,80.0,30.726645,True


In [28]:
#use a lambda expression
df.groupby('Sex')['Age'].agg(['max', 'mean', lambda x: True if x.mean() > 50 else False])

,max,mean,<lambda>
Sex,,,
female,63.0,27.915709,False
male,80.0,30.726645,False


### Without column

In [30]:
#Without a column, it will perform the aggregation across all of the numeric columns
df.groupby('Sex').mean()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Sex,,,,,,,
female,431.028662,0.742038,2.159236,27.915709,0.694268,0.649682,44.479818
male,454.147314,0.188908,2.389948,30.726645,0.429809,0.235702,25.523893


In [31]:
df.groupby('Sex').agg(['max', 'min', 'count', 'median', 'mean','sum'])

PassengerId                                      Survived            \
               max min count median        mean     sum      max min count   
Sex                                                                          
female         889   2   314  414.5  431.028662  135343        1   0   314   
male           891   1   577  464.0  454.147314  262043        1   0   577   

                  ...     Parch                            Fare              \
       median     ...     count median      mean  sum       max   min count   
Sex               ...                                                         
female      1     ...       314      0  0.649682  204  512.3292  6.75   314   
male        0     ...       577      0  0.235702  136  512.3292  0.00   577   

                                      
       median       mean         sum  
Sex                                   
female   23.0  44.479818  13966.6628  
male     10.5  25.523893  14727.2865  

[2 rows x 42 columns]

# 3. Transforming data

Transformation is a process in which we perform some group-specific computations and return a like-indexed (same length) object. When looking for transforming data, transform() and apply() are the most commonly used functions.

In [32]:
#lambda expression for Standardization.
standardization = lambda x: (x - x.mean()) / x.std()

In [33]:
#standardization on Age column with transform()
df.groupby('Sex')['Age'].transform(standardization)

0     -0.594531
1      0.714684
2     -0.135768
3      0.502071
4      0.291136
5           NaN
6      1.585573
7     -1.957096
8     -0.064897
9     -0.986220
10    -1.694930
11     2.132103
12    -0.730787
13     0.563649
14    -0.986220
15     1.919490
16    -1.957096
17          NaN
18     0.218587
19          NaN
20     0.291136
21     0.223008
22    -0.915349
23    -0.185761
24    -1.411446
25     0.714684
26          NaN
27    -0.798916
28          NaN
29          NaN
         ...   
861   -0.662659
862    1.423394
863         NaN
864   -0.458274
865    0.998168
866   -0.064897
867    0.018623
868         NaN
869   -1.820839
870   -0.322018
871    1.352523
872    0.154880
873    1.108675
874    0.005974
875   -0.915349
876   -0.730787
877   -0.798916
878         NaN
879    1.990361
880   -0.206639
881    0.154880
882   -0.419252
883   -0.185761
884   -0.390146
885    0.785555
886   -0.253890
887   -0.631865
888         NaN
889   -0.322018
890    0.086751
Name: Age, Length: 891, 

In [34]:
df.groupby('Sex')['Age'].apply(standardization)

0     -0.594531
1      0.714684
2     -0.135768
3      0.502071
4      0.291136
5           NaN
6      1.585573
7     -1.957096
8     -0.064897
9     -0.986220
10    -1.694930
11     2.132103
12    -0.730787
13     0.563649
14    -0.986220
15     1.919490
16    -1.957096
17          NaN
18     0.218587
19          NaN
20     0.291136
21     0.223008
22    -0.915349
23    -0.185761
24    -1.411446
25     0.714684
26          NaN
27    -0.798916
28          NaN
29          NaN
         ...   
861   -0.662659
862    1.423394
863         NaN
864   -0.458274
865    0.998168
866   -0.064897
867    0.018623
868         NaN
869   -1.820839
870   -0.322018
871    1.352523
872    0.154880
873    1.108675
874    0.005974
875   -0.915349
876   -0.730787
877   -0.798916
878         NaN
879    1.990361
880   -0.206639
881    0.154880
882   -0.419252
883   -0.185761
884   -0.390146
885    0.785555
886   -0.253890
887   -0.631865
888         NaN
889   -0.322018
890    0.086751
Name: Age, Length: 891, 

In [35]:
df.groupby('Sex')['Age'].apply(lambda x:(x-x.mean())/x.std())

0     -0.594531
1      0.714684
2     -0.135768
3      0.502071
4      0.291136
5           NaN
6      1.585573
7     -1.957096
8     -0.064897
9     -0.986220
10    -1.694930
11     2.132103
12    -0.730787
13     0.563649
14    -0.986220
15     1.919490
16    -1.957096
17          NaN
18     0.218587
19          NaN
20     0.291136
21     0.223008
22    -0.915349
23    -0.185761
24    -1.411446
25     0.714684
26          NaN
27    -0.798916
28          NaN
29          NaN
         ...   
861   -0.662659
862    1.423394
863         NaN
864   -0.458274
865    0.998168
866   -0.064897
867    0.018623
868         NaN
869   -1.820839
870   -0.322018
871    1.352523
872    0.154880
873    1.108675
874    0.005974
875   -0.915349
876   -0.730787
877   -0.798916
878         NaN
879    1.990361
880   -0.206639
881    0.154880
882   -0.419252
883   -0.185761
884   -0.390146
885    0.785555
886   -0.253890
887   -0.631865
888         NaN
889   -0.322018
890    0.086751
Name: Age, Length: 891, 

# 4. Filtration

Filtration is a process in which we discard some groups, according to a group-wise computation that evaluates True or False.

In [38]:
#filter data to return all passengers that lived in a cabin has ≥ 4 people.
df.groupby('Cabin').filter(lambda x:len(x)>=4).sort_values('Cabin')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
390,391,1,1,"Carter, Mr. William Ernest",male,36.0,1,2,113760,120.0000,B96 B98,S
435,436,1,1,"Carter, Miss. Lucile Polk",female,14.0,1,2,113760,120.0000,B96 B98,S
763,764,1,1,"Carter, Mrs. William Ernest (Lucile Polk)",female,36.0,1,2,113760,120.0000,B96 B98,S
802,803,1,1,"Carter, Master. William Thornton II",male,11.0,1,2,113760,120.0000,B96 B98,S
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0000,C23 C25 C27,S
341,342,1,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.0000,C23 C25 C27,S
438,439,0,1,"Fortune, Mr. Mark",male,64.0,1,4,19950,263.0000,C23 C25 C27,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
205,206,0,3,"Strom, Miss. Telma Matilda",female,2.0,0,1,347054,10.4625,G6,S


# 6. Grouping by multiple categories

In [41]:
# Creating a subset
df_subset = df.loc[:, ['Sex', 'Pclass', 'Age', 'Fare']]


# Group by multiple categories
df_subset.groupby(['Sex', 'Pclass']).mean()

Age        Fare
Sex    Pclass                       
female 1       34.611765  106.125798
       2       28.722973   21.970121
       3       21.750000   16.118810
male   1       41.281386   67.226127
       2       30.740707   19.741782
       3       26.507589   12.661633

# 7. Resetting index with as_index

In [43]:
df_groupby_multi = df_subset.groupby(['Sex', 'Pclass']).mean()
# Resetting index
df_groupby_multi.reset_index()

,Sex,Pclass,Age,Fare
0,female,1,34.611765,106.125798
1,female,2,28.722973,21.970121
2,female,3,21.750000,16.118810
3,male,1,41.281386,67.226127
4,male,2,30.740707,19.741782
5,male,3,26.507589,12.661633


In [44]:
## other way
df_subset.groupby(['Sex', 'Pclass'], as_index=False).mean()

,Sex,Pclass,Age,Fare
0,female,1,34.611765,106.125798
1,female,2,28.722973,21.970121
2,female,3,21.750000,16.118810
3,male,1,41.281386,67.226127
4,male,2,30.740707,19.741782
5,male,3,26.507589,12.661633


In [45]:
##ASHIS KUMAR SAHU